In [1]:
# =============================================================================
# 1. 필요한 라이브러리 및 예제 코드 다운로드 섹션
# =============================================================================

import urllib.request  # 웹에서 파일을 다운로드하기 위한 라이브러리
import zipfile         # ZIP 파일을 압축 해제하기 위한 라이브러리
import os              # 운영체제와 상호작용하기 위한 라이브러리 (파일/폴더 관리)
import sys             # 시스템 관련 기능을 사용하기 위한 라이브러리

# TensorFlow 예제 코드가 있는 GitHub 저장소의 ZIP 파일 URL
repo_url = "https://github.com/tensorflow/examples/archive/refs/heads/master.zip"

# urllib.request.urlretrieve(): 웹에서 파일을 다운로드하여 로컬에 저장
# repo_url에서 파일을 다운로드하여 "tensorflow_examples.zip"이라는 이름으로 저장
urllib.request.urlretrieve(repo_url, "tensorflow_examples.zip")

# with문을 사용하여 ZIP 파일을 안전하게 열고 자동으로 닫기
# zipfile.ZipFile(): ZIP 파일을 읽기 모드("r")로 열기
with zipfile.ZipFile("tensorflow_examples.zip", "r") as zip_ref:
   # extractall("."): 현재 디렉토리(".")에 ZIP 파일의 모든 내용을 압축 해제
   zip_ref.extractall(".")

# 기존에 "examples" 폴더가 있는지 확인
if os.path.exists("examples"):
   # 기존 폴더가 있다면 삭제 (rm -rf examples: 강제로 폴더와 내용 모두 삭제)
   os.system("rm -rf examples")

# 압축 해제된 폴더명 "examples-master"를 "examples"로 변경
os.rename("examples-master", "examples")

# sys.path.append(): Python이 모듈을 찾을 수 있는 경로에 "./examples" 추가
# 이렇게 하면 examples 폴더 안의 모듈들을 import할 수 있게 됨
sys.path.append("./examples")

# TensorFlow 예제에서 pix2pix 모델을 import
# pix2pix: 이미지를 다른 형태의 이미지로 변환하는 GAN 모델
from tensorflow_examples.models.pix2pix import pix2pix

# =============================================================================
# 2. 필요한 라이브러리들 import
# =============================================================================

import tensorflow as tf                    # 딥러닝 프레임워크 TensorFlow
import tensorflow_datasets as tfds         # TensorFlow에서 제공하는 다양한 데이터셋 라이브러리
from tensorflow_examples.models.pix2pix import pix2pix  # pix2pix 모델 (위에서 이미 import했지만 명시적으로 다시)
import os                                 # 운영체제 관련 기능
import time                               # 시간 측정을 위한 라이브러리
import matplotlib.pyplot as plt           # 그래프와 이미지를 시각화하기 위한 라이브러리
from IPython.display import clear_output  # Jupyter 노트북에서 출력을 지우기 위한 기능

# AUTOTUNE: TensorFlow가 자동으로 최적의 병렬 처리 수를 결정하도록 하는 상수
AUTOTUNE = tf.data.AUTOTUNE

# =============================================================================
# 3. 데이터셋 로드 및 전처리 설정
# =============================================================================

# tfds.load(): TensorFlow Datasets에서 CycleGAN용 horse2zebra 데이터셋 로드
# 'cycle_gan/horse2zebra': 말과 얼룩말 이미지가 쌍을 이루지 않는 데이터셋
# with_info=True: 데이터셋의 메타데이터 정보도 함께 반환
# as_supervised=True: (이미지, 라벨) 형태로 데이터를 반환 (하지만 CycleGAN에서는 라벨을 사용하지 않음)
dataset, metadata = tfds.load('cycle_gan/horse2zebra',
                              with_info=True, as_supervised=True)

# 데이터셋을 학습용과 테스트용으로 분리
# trainA: 말 이미지 학습 데이터, trainB: 얼룩말 이미지 학습 데이터
train_horses, train_zebras = dataset['trainA'], dataset['trainB']
# testA: 말 이미지 테스트 데이터, testB: 얼룩말 이미지 테스트 데이터
test_horses, test_zebras = dataset['testA'], dataset['testB']

# 하이퍼파라미터 설정
BUFFER_SIZE = 1000  # 데이터를 섞을 때 사용할 버퍼 크기 (메모리에 1000개 이미지를 저장해두고 랜덤하게 섞음)
BATCH_SIZE = 1      # 한 번에 처리할 이미지의 개수 (1개씩 처리)
IMG_WIDTH = 256     # 이미지의 너비를 256픽셀로 설정
IMG_HEIGHT = 256    # 이미지의 높이를 256픽셀로 설정

# =============================================================================
# 4. 이미지 전처리 함수들 정의
# =============================================================================

def normalize(image):
    """
    이미지를 정규화하는 함수
    픽셀 값을 [0, 255] 범위에서 [-1, 1] 범위로 변환
    """
    # tf.cast(): 이미지의 데이터 타입을 float32로 변환 (연산 효율성을 위해)
    image = tf.cast(image, tf.float32)
    # 픽셀 값을 127.5로 나누고 1을 빼서 [-1, 1] 범위로 정규화
    # (0~255) / 127.5 - 1 = (0~2) - 1 = (-1~1)
    image = (image / 127.5) - 1
    return image

"""# 추가된 함수 (책에는 빠져있으니 화면 보고 작성해주세요)"""

def random_crop(image):
    """
    이미지에서 랜덤한 위치의 부분을 잘라내는 함수 (데이터 증강 기법)
    """
    # tf.image.random_crop(): 이미지에서 [IMG_HEIGHT, IMG_WIDTH, 3] 크기의 부분을 랜덤하게 자르기
    # 3은 RGB 채널 수
    return tf.image.random_crop(image, size=[IMG_HEIGHT, IMG_WIDTH, 3])

"""# 아래 코드 내용을 화면이나 책을 보고 채워주세요"""

def random_jitter(image):
    """
    이미지에 랜덤한 변형을 가하는 함수 (데이터 증강)
    크기 조정 → 랜덤 자르기 → 좌우 반전을 순서대로 적용
    """
    # tf.image.resize(): 이미지 크기를 286x286으로 확대
    # ResizeMethod.NEAREST_NEIGHBOR: 가장 가까운 이웃 픽셀 값으로 보간 (빠르지만 품질은 낮음)
    image = tf.image.resize(image, [286, 286],
                          method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)

    # 286x286 이미지에서 256x256 부분을 랜덤하게 자르기
    image = random_crop(image)

    # tf.image.random_flip_left_right(): 50% 확률로 이미지를 좌우 반전
    image = tf.image.random_flip_left_right(image)

    return image

"""# 아래 코드 내용을 화면이나 책을 보고 채워주세요"""

def preprocess_image_train(image, label):
    """
    학습용 이미지 전처리 함수
    데이터 증강(random_jitter)과 정규화를 함께 적용
    """
    # 랜덤한 변형 적용 (데이터 증강으로 모델의 일반화 성능 향상)
    image = random_jitter(image)
    # 픽셀 값 정규화 ([-1, 1] 범위로 변환)
    image = normalize(image)
    # label은 사용하지 않지만 함수 시그니처를 맞추기 위해 받음
    return image

def preprocess_image_test(image, label):
    """
    테스트용 이미지 전처리 함수
    데이터 증강 없이 정규화만 적용 (일관된 결과를 위해)
    """
    # 정규화만 적용 (테스트 시에는 일관된 결과를 위해 랜덤 변형 적용하지 않음)
    image = normalize(image)
    return image

"""# 아래 코드 내용을 화면이나 책을 보고 채워주세요"""

# =============================================================================
# 5. 데이터셋 전처리 파이프라인 구성
# =============================================================================

# 학습용 데이터 전처리 파이프라인 (정상 순서)
# .map(): 각 이미지에 전처리 함수를 적용
# num_parallel_calls=AUTOTUNE: 병렬 처리로 전처리 속도 향상
train_horses = train_horses.map(preprocess_image_train, num_parallel_calls=AUTOTUNE)
# .cache(): 전처리된 데이터를 메모리에 캐시하여 다음 epoch에서 빠르게 접근
# .shuffle(): 데이터 순서를 랜덤하게 섞어서 학습 효과 향상
# .batch(): 지정된 배치 크기로 데이터를 묶음
train_horses = train_horses.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

# 얼룩말 학습 데이터도 동일하게 전처리
train_zebras = train_zebras.map(preprocess_image_train, num_parallel_calls=AUTOTUNE)
train_zebras = train_zebras.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

# 테스트용 데이터 전처리 (shuffle 제거, cache는 optional)
# 테스트 데이터는 shuffle하지 않음 (일관된 평가를 위해)
test_horses = test_horses.map(preprocess_image_test, num_parallel_calls=AUTOTUNE)
test_horses = test_horses.cache().batch(BATCH_SIZE)

test_zebras = test_zebras.map(preprocess_image_test, num_parallel_calls=AUTOTUNE)
test_zebras = test_zebras.cache().batch(BATCH_SIZE)

# 시각화를 위한 샘플 이미지 추출
# next(iter()): 데이터셋에서 첫 번째 배치를 가져옴
sample_horse = next(iter(train_horses))
sample_zebra = next(iter(train_zebras))

"""# 아래 코드 내용을 화면이나 책을 보고 채워주세요"""

# =============================================================================
# 6. 데이터 증강 효과 시각화
# =============================================================================

# subplot(121): 1행 2열 구조에서 첫 번째 위치
plt.subplot(121)
plt.title('Horse')  # 제목 설정
# 정규화된 이미지를 다시 [0, 1] 범위로 변환하여 시각화
# sample_horse[0]: 배치의 첫 번째 이미지
# * 0.5 + 0.5: [-1, 1] 범위를 [0, 1] 범위로 변환
plt.imshow(sample_horse[0] * 0.5 + 0.5)

# subplot(122): 1행 2열 구조에서 두 번째 위치
plt.subplot(122)
plt.title('Horse with random jitter')  # 데이터 증강이 적용된 이미지
# random_jitter 함수를 적용하여 변형된 이미지 시각화
plt.imshow(random_jitter(sample_horse[0]) * 0.5 + 0.5)

"""# 아래 코드 내용을 화면이나 책을 보고 채워주세요"""

# 얼룩말 이미지에 대해서도 동일한 시각화 수행
plt.subplot(121)
plt.title('Zebra')
plt.imshow(sample_zebra[0] * 0.5 + 0.5)

plt.subplot(122)
plt.title('Zebra with random jitter')
plt.imshow(random_jitter(sample_zebra[0]) * 0.5 + 0.5)

"""# 아래 코드 내용을 화면이나 책을 보고 채워주세요"""

# =============================================================================
# 7. CycleGAN 모델 구성 요소 정의
# =============================================================================

OUTPUT_CHANNELS = 3  # 출력 이미지의 채널 수 (RGB이므로 3)

# Generator 모델들 생성
# unet_generator: U-Net 구조의 생성자 모델 (이미지를 다른 도메인으로 변환)
# norm_type='instancenorm': Instance Normalization 사용 (배치 정규화보다 스타일 변환에 효과적)
generator_g = pix2pix.unet_generator(OUTPUT_CHANNELS, norm_type='instancenorm')  # 말 → 얼룩말 변환기
generator_f = pix2pix.unet_generator(OUTPUT_CHANNELS, norm_type='instancenorm')  # 얼룩말 → 말 변환기

# Discriminator 모델들 생성
# discriminator: 입력 이미지가 진짜인지 가짜인지 판별하는 모델
# target=False: pix2pix와 달리 조건부 입력이 없음을 의미
discriminator_x = pix2pix.discriminator(norm_type='instancenorm', target=False)  # 말 이미지 판별기
discriminator_y = pix2pix.discriminator(norm_type='instancenorm', target=False)  # 얼룩말 이미지 판별기

# =============================================================================
# 8. 손실 함수 및 하이퍼파라미터 설정
# =============================================================================

LAMBDA = 10  # Cycle Consistency Loss의 가중치 (얼마나 원본과 비슷하게 복원되어야 하는지)

# Binary Cross Entropy 손실 함수 정의
# from_logits=True: 모델 출력이 시그모이드를 거치지 않은 raw logits임을 의미
loss_obj = tf.keras.losses.BinaryCrossentropy(from_logits=True)

"""# 아래 코드 내용을 화면이나 책을 보고 채워주세요"""

# =============================================================================
# 9. 손실 함수들 정의
# =============================================================================

def discriminator_loss(real, generated):
    """
    Discriminator의 손실을 계산하는 함수
    진짜 이미지는 1로, 가짜 이미지는 0으로 분류하도록 학습
    """
    # 진짜 이미지에 대한 손실: 1이라고 예측해야 함
    # tf.ones_like(real): real과 같은 shape의 1로 채워진 텐서 생성
    real_loss = loss_obj(tf.ones_like(real), real)

    # 가짜 이미지에 대한 손실: 0이라고 예측해야 함
    # tf.zeros_like(generated): generated와 같은 shape의 0으로 채워진 텐서 생성
    generated_loss = loss_obj(tf.zeros_like(generated), generated)

    # 전체 Discriminator 손실: 진짜와 가짜 손실의 합
    total_disc_loss = real_loss + generated_loss

    # 0.5를 곱하는 이유: Generator와 Discriminator의 학습 균형을 맞추기 위해
    return total_disc_loss * 0.5

def generator_loss(generated):
    """
    Generator의 기본 손실 함수
    생성된 이미지가 Discriminator를 속여서 1(진짜)로 분류되도록 하는 것이 목표
    """
    # Generator는 Discriminator가 생성된 이미지를 1(진짜)로 분류하기를 원함
    return loss_obj(tf.ones_like(generated), generated)

def calc_cycle_loss(real_image, cycled_image):
    """
    Cycle Consistency Loss 계산
    원본 이미지와 cycle을 거쳐 복원된 이미지 간의 차이를 측정
    예: 말 → 얼룩말 → 말 과정에서 원본 말과 최종 말이 얼마나 비슷한지
    """
    # tf.reduce_mean(): 전체 픽셀에 대한 평균 계산
    # tf.abs(): 절댓값 (L1 loss 사용, L2 loss보다 선명한 결과 생성)
    loss1 = tf.reduce_mean(tf.abs(real_image - cycled_image))

    # LAMBDA를 곱해서 cycle consistency의 중요도 조절
    return LAMBDA * loss1

def identity_loss(real_image, same_image):
    """
    Identity Loss 계산
    같은 도메인의 이미지를 입력했을 때 변화가 없어야 함을 보장
    예: 얼룩말을 얼룩말 생성기에 넣었을 때 원본과 같아야 함
    """
    # 원본과 동일 도메인 변환 결과 간의 차이 계산
    loss = tf.reduce_mean(tf.abs(real_image - same_image))

    # LAMBDA * 0.5: identity loss는 cycle loss보다 상대적으로 적은 가중치
    return LAMBDA * 0.5 * loss

"""# 아래 코드 내용을 화면이나 책을 보고 채워주세요"""

# =============================================================================
# 10. 옵티마이저 설정
# =============================================================================

# Adam 옵티마이저 설정 (적응적 학습률 조정 알고리즘)
# 2e-4: 학습률 0.0002 (GAN 학습에 일반적으로 사용되는 값)
# beta_1=0.5: 모멘텀 계수 (기본값 0.9보다 낮게 설정하여 GAN 학습 안정화)
generator_g_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
generator_f_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)

# 학생이 채워야 할 부분 (원본 코드에서 비어있음)
# Discriminator용 옵티마이저도 동일한 설정으로 생성해야 함
discriminator_x_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
discriminator_y_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)

# =============================================================================
# 11. 체크포인트 설정 (모델 저장/복원)
# =============================================================================

checkpoint_path = "./checkpoints/train"  # 체크포인트 저장 경로

# tf.train.Checkpoint: 모델의 가중치와 옵티마이저 상태를 저장/복원하는 객체
ckpt = tf.train.Checkpoint(generator_g=generator_g,                    # Generator G 모델
                           generator_f=generator_f,                    # Generator F 모델
                           discriminator_x=discriminator_x,            # Discriminator X 모델
                           discriminator_y=discriminator_y,            # Discriminator Y 모델
                           generator_g_optimizer=generator_g_optimizer,     # Generator G 옵티마이저
                           generator_f_optimizer=generator_f_optimizer,     # Generator F 옵티마이저
                           discriminator_x_optimizer=discriminator_x_optimizer,  # Discriminator X 옵티마이저
                           discriminator_y_optimizer=discriminator_y_optimizer)  # Discriminator Y 옵티마이저

# CheckpointManager: 체크포인트 파일 관리 (최대 5개까지 유지)
ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

"""# 아래 코드 내용을 화면이나 책을 보고 채워주세요"""

# 기존 체크포인트가 있다면 복원
if ckpt_manager.latest_checkpoint:
    # 가장 최근 체크포인트에서 모델과 옵티마이저 상태 복원
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print('Latest checkpoint restored!!')

EPOCHS = 50  # 전체 학습 에포크 수

"""# 아래 코드 내용을 화면이나 책을 보고 채워주세요"""

# =============================================================================
# 12. 시각화 및 학습 함수들 정의
# =============================================================================

def generate_images(model, test_input):
    """
    모델을 사용하여 이미지를 생성하고 시각화하는 함수
    """
    # 모델에 테스트 입력을 넣어 예측 결과 생성
    prediction = model(test_input)

    # 12x12 크기의 figure 생성
    plt.figure(figsize=(12, 12))

    # 입력 이미지와 예측 결과를 리스트로 구성
    display_list = [test_input[0], prediction[0]]  # 배치의 첫 번째 이미지들
    title = ['Input Image', 'Predicted Image']     # 각 이미지의 제목

    # 2개 이미지를 나란히 표시
    for i in range(2):
        plt.subplot(1, 2, i+1)          # 1행 2열에서 i+1번째 위치
        plt.title(title[i])             # 제목 설정
        # 정규화된 이미지를 [0, 1] 범위로 변환하여 표시
        plt.imshow(display_list[i] * 0.5 + 0.5)
        plt.axis('off')                 # 축 정보 숨기기
    plt.show()

def train_step(real_x, real_y):
    """
    CycleGAN의 한 번의 학습 스텝을 수행하는 함수
    real_x: 실제 말 이미지, real_y: 실제 얼룩말 이미지
    """
    # GradientTape: 자동 미분을 위한 컨텍스트 매니저
    # persistent=True: 여러 번 gradient를 계산할 수 있도록 설정
    with tf.GradientTape(persistent=True) as tape:

        # =============================================================================
        # Forward Pass: 이미지 생성 및 판별
        # =============================================================================

        # Generator G: 말 → 얼룩말 변환
        fake_y = generator_g(real_x, training=True)
        # Generator F를 사용해 변환된 얼룩말을 다시 말로 복원 (Cycle)
        cycled_x = generator_f(fake_y, training=True)

        # Generator F: 얼룩말 → 말 변환
        fake_x = generator_f(real_y, training=True)
        # Generator G를 사용해 변환된 말을 다시 얼룩말로 복원 (Cycle)
        cycled_y = generator_g(fake_x, training=True)

        # Identity Mapping: 같은 도메인 이미지 변환 (변화가 없어야 함)
        same_x = generator_f(real_x, training=True)  # 말을 말 생성기에 넣기
        same_y = generator_g(real_y, training=True)  # 얼룩말을 얼룩말 생성기에 넣기

        # Discriminator들의 판별 결과
        disc_real_x = discriminator_x(real_x, training=True)    # 진짜 말에 대한 판별
        disc_real_y = discriminator_y(real_y, training=True)    # 진짜 얼룩말에 대한 판별
        disc_fake_x = discriminator_x(fake_x, training=True)    # 가짜 말에 대한 판별
        disc_fake_y = discriminator_y(fake_y, training=True)    # 가짜 얼룩말에 대한 판별

        # =============================================================================
        # Loss 계산
        # =============================================================================

        # Generator Loss: Discriminator를 속이기 위한 손실
        gen_g_loss = generator_loss(disc_fake_y)  # Generator G가 만든 가짜 얼룩말이 진짜로 판별되도록
        gen_f_loss = generator_loss(disc_fake_x)  # Generator F가 만든 가짜 말이 진짜로 판별되도록

        # Cycle Loss: 원본 → 변환 → 복원 과정에서의 일관성 손실
        total_cycle_loss = calc_cycle_loss(real_x, cycled_x) + calc_cycle_loss(real_y, cycled_y)

        # 전체 Generator Loss
        # Generator G (말 → 얼룩말) 방향의 전체 손실 계산
        # 1. Generator가 만든 얼룩말 이미지가 진짜처럼 보이도록 속였는가? → gen_g_loss
        # 2. 말 → 얼룩말 → 다시 말로 되돌렸을 때 원래 말 이미지와 얼마나 비슷한가? → total_cycle_loss
        # 3. 얼룩말을 얼룩말로 변환했을 때 원본과 같은가? → identity_loss
        # 세 가지 손실을 모두 더해서 Generator G의 학습 기준을 정합니다.
        total_gen_g_loss = gen_g_loss + total_cycle_loss + identity_loss(real_y, same_y)

        # Generator F (얼룩말 → 말) 방향의 전체 손실 계산
        # 1. Generator가 만든 말 이미지가 진짜처럼 보이도록 속였는가? → gen_f_loss
        # 2. 얼룩말 → 말 → 다시 얼룩말로 되돌렸을 때 원래 얼룩말과 얼마나 비슷한가? → total_cycle_loss
        # 3. 말 이미지를 그대로 말로 변환했을 때, 원본과 같은가? → identity_loss
        # 세 가지 손실을 모두 더해서 Generator F의 학습 기준을 정합니다.
        total_gen_f_loss = gen_f_loss + total_cycle_loss + identity_loss(real_x, same_x)

        # Discriminator Loss
        # Discriminator 손실 계산 (말 방향)
        # disc_real_x: 진짜 말 이미지에 대한 Discriminator의 출력
        # disc_fake_x: 가짜 말 이미지 (Generator가 만든 것)에 대한 Discriminator의 출력
        # 이 두 값을 가지고 Discriminator가 잘 구별했는지를 평가합니다.
        disc_x_loss = discriminator_loss(disc_real_x, disc_fake_x)

        # Discriminator 손실 계산 (얼룩말 방향)
        # disc_real_y: 진짜 얼룩말
        # disc_fake_y: Generator가 만든 가짜 얼룩말
        disc_y_loss = discriminator_loss(disc_real_y, disc_fake_y)

    # =============================================================================
    # Gradient 계산 및 모델 업데이트
    # =============================================================================

    # Gradient 계산
    # tape.gradient(): 손실에 대한 각 모델 파라미터의 기울기 계산
    generator_g_gradients = tape.gradient(total_gen_g_loss, generator_g.trainable_variables)  # 이게 얼마나 틀렸는지를 기준으로 Generator의 파라미터를 어떻게 바꿀지 정합니다
    generator_f_gradients = tape.gradient(total_gen_f_loss, generator_f.trainable_variables)  # Generator F의 손실 기반 Gradient
    discriminator_x_gradients = tape.gradient(disc_x_loss, discriminator_x.trainable_variables)  # Discriminator X의 손실(disc_x_loss)에 대한 Gradient
    discriminator_y_gradients = tape.gradient(disc_y_loss, discriminator_y.trainable_variables)  # Discriminator Y의 손실에 대한 Gradient

    # Optimizer로 업데이트 -> 추가된 부분
    # 앞서 계산한 Gradient를 가지고 실제로 '가중치'를 수정하여 학습이 일어나게 합니다.
    # zip(): gradient와 해당하는 변수들을 쌍으로 묶어줌
    # apply_gradients(): 계산된 기울기를 사용하여 실제 파라미터 업데이트
    generator_g_optimizer.apply_gradients(zip(generator_g_gradients, generator_g.trainable_variables))      # Generator G 업데이트
    generator_f_optimizer.apply_gradients(zip(generator_f_gradients, generator_f.trainable_variables))      # Generator F 업데이트
    discriminator_x_optimizer.apply_gradients(zip(discriminator_x_gradients, discriminator_x.trainable_variables))  # Discriminator X 업데이트
    discriminator_y_optimizer.apply_gradients(zip(discriminator_y_gradients, discriminator_y.trainable_variables))  # Discriminator Y 업데이트

"""# 아래 코드 내용을 화면이나 책을 보고 채워주세요"""

# =============================================================================
# 13. 실제 학습 루프
# =============================================================================

MAX_BATCHES = 1  # 한 에포크당 처리할 최대 배치 수 (실제로는 더 많이 해야 하지만 예제용)

# 전체 에포크에 대해 반복
for epoch in range(EPOCHS):
    start = time.time()  # 에포크 시작 시간 기록
    n = 0               # 현재 배치 번호

    # tf.data.Dataset.zip(): 말과 얼룩말 데이터셋을 쌍으로 묶음
    # .take(MAX_BATCHES): 지정된 수만큼만 배치 처리
    for image_x, image_y in tf.data.Dataset.zip((train_horses, train_zebras)).take(MAX_BATCHES):
        # 한 스텝의 학습 수행
        train_step(image_x, image_y)

        # 100번마다 진행 상황 표시
        if n % 100 == 0:
            print('.', end='')  # 줄바꿈 없이 점 출력
        n += 1

    # clear_output(wait=True): 이전 출력을 지우고 새로운 결과만 표시
    clear_output(wait=True)

    # 현재 모델로 샘플 이미지 생성하여 진행 상황 확인
    generate_images(generator_g, sample_horse)

    # 에포크 완료 시간 출력
    print(f"\nEpoch {epoch+1} completed in {time.time()-start:.2f} sec")

"""# 아래 코드 내용을 화면이나 책을 보고 채워주세요"""

    # 5 에포크마다 체크포인트 저장 (실제로는 for 루프 안에 있어야 함)
    if (epoch + 1) % 5 == 0:
        # 현재 모델 상태를 체크포인트로 저장
        ckpt_save_path = ckpt_manager.save()
        print('Saving checkpoint for epoch {} at {}'.format(epoch+ 1, ckpt_save_path))
        print('Time taken for epoch {} is {} sec\n'.format(epoch + 1, time.time()-start))

"""# 아래 코드 내용을 화면이나 책을 보고 채워주세요"""

# =============================================================================
# 14. 테스트 결과 확인
# =============================================================================

# 테스트 말 이미지 5개에 대해 얼룩말 변환 결과 확인
for inp in test_horses.take(5):
    generate_images(generator_g, inp)

"""# 아래 코드 내용을 화면이나 책을 보고 채워주세요"""

# =============================================================================
# 15. 최종 결과 시각화 및 Discriminator 동작 확인
# =============================================================================

# 샘플 이미지들을 서로 변환
to_zebra = generator_g(sample_horse)  # 말을 얼룩말로 변환
to_horse = generator_f(sample_zebra)  # 얼룩말을 말로 변환

plt.figure(figsize=(8, 8))
# contrast = 8  # 주석 처리된 부분

# 원본과 변환 결과들을 하나의 리스트로 구성
imgs = [sample_horse, to_zebra, sample_zebra, to_horse]
title = ['Horse', 'To Zebra', 'Zebra', 'To Horse']

# 4개 이미지를 2x2 격자로 표시 (학생이 완성해야 할 부분)
for i in range(len(imgs)):
    plt.subplot(2, 2, i+1)  # 2x2 격자에서 i+1번째 위치
    plt.title(title[i])     # 각 이미지의 제목
    plt.imshow(imgs[i][0] * 0.5 + 0.5)  # 정규화 해제하여 표시
    plt.axis('off')         # 축 정보 숨기기

plt.show()

# Discriminator의 판별 결과 시각화
plt.figure(figsize=(8, 8))

# 생성된 얼룩말이 진짜 얼룩말로 판별되는지 확인
plt.subplot(121)
plt.title('Is a real zebra?')
# discriminator_y(to_zebra): 생성된 얼룩말에 대한 판별 결과
# [0, ..., -1]: 배치의 첫 번째 이미지의 마지막 채널
# cmap='gray': 그레이스케일로 표시 (높은 값일수록 진짜로 판별)
plt.imshow(discriminator_y(to_zebra)[0, ..., -1], cmap='gray')

# 생성된 말이 진짜 말로 판별되는지 확인
plt.subplot(122)
plt.title('Is a real horse?')
plt.imshow(discriminator_x(to_horse)[0, ..., -1], cmap='gray')
plt.show()

IndentationError: unexpected indent (ipython-input-1-1987021527.py, line 499)